In [21]:
import pandas as pd
import uuid
import re
import random
from datetime import datetime, timedelta
import numpy as np

Data_Frame = pd.read_csv("medicines_Egyptian_data_2024.csv")

print("✅ Columns:", list(Data_Frame.columns))
print("\nData Info:")
print(Data_Frame.info())
Data_Frame.dropna(inplace=True)
print(f"\n✅ After dropping empty/null rows, remaining rows: {len(Data_Frame)}")

# Helper functions
def guid():
    return str(uuid.uuid4())

def random_date():
    return datetime.now() + timedelta(days=random.randint(30, 730))

✅ Columns: ['Active_Ingredient', 'company', 'created', 'form', 'group', 'id', 'Price', 'pharmacology', 'route', 'Name', 'updated']

Data Info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 26201 entries, 0 to 26200
Data columns (total 11 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Active_Ingredient  23292 non-null  object 
 1   company            22517 non-null  object 
 2   created            26201 non-null  object 
 3   form               26189 non-null  object 
 4   group              22061 non-null  object 
 5   id                 26201 non-null  object 
 6   Price              26199 non-null  float64
 7   pharmacology       13319 non-null  object 
 8   route              25427 non-null  object 
 9   Name               26201 non-null  object 
 10  updated            26201 non-null  object 
dtypes: float64(1), object(10)
memory usage: 2.2+ MB
None

✅ After dropping empty/null rows, remaining rows: 11671


In [40]:
# Extract unique groups From Data
unique_groups = Data_Frame["group"].dropna().unique()
unique_groups = [x[:50] if isinstance(x, str) else x for x in unique_groups]


category_map = {g: guid() for g in unique_groups}

df_categories = pd.DataFrame({
    "Id": list(category_map.values()),
    "Name": list(category_map.keys()),
    "LogoUrl": "https://res.cloudinary.com/dwahjkgbk/image/upload/v1761570572/Screenshot_20251027_160640_Chrome_dd8usi.jpg",
    "IsDeleted": 0,
    "ProductsCount": [Data_Frame["group"].value_counts().get(name, 0) for name in category_map.keys()]
})
df_categories = df_categories[df_categories["Id"] != "Id"]
df_categories = df_categories.drop_duplicates(subset=["Name"])
df_categories.to_csv("Categories.csv", index=False)
print(f"✅ Categories.csv created with {len(df_categories)} rows")

✅ Categories.csv created with 1204 rows


In [41]:
# Extract unique companies and their IDs
unique_companies = Data_Frame["company"].dropna().unique()

company_map = {c: guid() for c in unique_companies}

df_companies = pd.DataFrame({
    "Id": list(company_map.values()),
    "Name": list(company_map.keys()),
    "LogoUrl": "https://res.cloudinary.com/dwahjkgbk/image/upload/v1761570652/Screenshot_20251027_160704_Chrome_lzsssz.jpg",
    "IsDeleted": 0,
    "ProductsCount": [Data_Frame["company"].value_counts().get(name, 0) for name in company_map.keys()]
})
df_companies = df_companies[df_companies["Id"] != "Id"]
df_companies = df_companies.drop_duplicates(subset=["Name"])
df_companies.to_csv("Companies.csv", index=False)
print(f"✅ Companies.csv created with {len(df_companies)} rows")


✅ Companies.csv created with 2702 rows


In [42]:
# Transform main DataFrame to match Product model
# from googletrans import Translator
# translator = Translator()

# def translate_to_arabic(text):
#     if pd.isna(text):
#         return None
#     try:
#         result = translator.translate(text, src='en', dest='ar')
#         return result.text
#     except Exception:
#         return text  # fallback if translation fails

products = pd.DataFrame({
    "ProductId": [guid() for _ in range(len(Data_Frame))],
    "NameAr" :Data_Frame["Name"],#.apply(translate_to_arabic),
    "NameEn": Data_Frame["Name"],
    "CategoryId": Data_Frame["group"].map(category_map),
    "CompanyId": Data_Frame["company"].map(company_map),
    "Description": Data_Frame["pharmacology"].fillna(Data_Frame["route"]),
    "MedicalDescription": Data_Frame["pharmacology"],
    "Tags": Data_Frame["group"],
    "AverageRating": [round(random.uniform(2, 5), 1) for _ in range(len(Data_Frame))],
    "TotalRatings": [random.randint(0, 500) for _ in range(len(Data_Frame))],
    "DiscountPercentage": [round(random.uniform(0, 0.3), 2) for _ in range(len(Data_Frame))],
    "ActiveIngredients": Data_Frame["Active_Ingredient"],
    "SideEffects": None,
    "Contraindications": None,
    "Price": Data_Frame["Price"].fillna(0).round(2),
    "IsDeleted": 0,
    "IsAvailable": Data_Frame["Price"].fillna(0) > 0,
    "SearchVector": None,
    "EmbeddingVector": None,
    "ExpirationDate": [random_date() for _ in range(len(Data_Frame))],
    "DosageForm": Data_Frame["form"],
})
# Function to remove illegal Excel characters
def clean_excel_text(value):
    if isinstance(value, str):
        return re.sub(r"[\x00-\x1F\x7F-\x9F]", "", value)
    return value
products = products.head(4500)
products = products.drop_duplicates()
# Apply cleaning to every column
products = products.applymap(clean_excel_text)
products.to_csv("Products.csv", index=False)
print(f"✅ Products.csv created successfully with {len(products)} products!")

✅ Products.csv created successfully with 4500 products!


C:\Users\NanoChip\AppData\Local\Temp\ipykernel_12920\2043917160.py:45: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  products = products.applymap(clean_excel_text)


In [28]:
default_image_url = "https://res.cloudinary.com/dwahjkgbk/image/upload/v1761570733/Screenshot_20251027_160539_Chrome_oh5dvy.jpg"

df_images = pd.DataFrame({
    "Id": [guid() for _ in range(len(products))],
    "Url": [default_image_url for _ in range(len(products))],
    "IsPrimary": 1,
    "ProductId": products["ProductId"]
})
df_images.to_csv("ProductImages.csv", index=False)
print(f"✅ ProductImages.csv created with {len(df_images)} images")

✅ ProductImages.csv created with 4500 images


In [26]:
store_names = [
    "SmartCare Downtown", "SmartCare Giza", "SmartCare Alexandria",
    "SmartCare Nasr City", "SmartCare Maadi", "SmartCare Tanta",
    "SmartCare Hurghada", "SmartCare Mansoura", "SmartCare Sohag",
    "SmartCare Aswan"
]

addresses = [
    "10 El Tahrir St, Cairo", "5 Pyramid Rd, Giza", "20 Corniche, Alexandria",
    "50 Abbas El Akkad, Nasr City", "15 Street 9, Maadi", "7 El Geish St, Tanta",
    "3 Sheraton Rd, Hurghada", "25 Al Gomhoria St, Mansoura", "12 El Kornaish St, Sohag",
    "30 Nile St, Aswan"
]

phones = ["0101" + str(random.randint(1000000, 9999999)) for _ in range(10)]
latitudes = [round(random.uniform(22.0, 31.0), 6) for _ in range(10)]
longitudes = [round(random.uniform(29.0, 33.0), 6) for _ in range(10)]

df_stores = pd.DataFrame({
    "Id": [guid() for _ in range(10)],
    "Name": store_names,
    "Address": addresses,
    "Latitude": latitudes,
    "Longitude": longitudes,
    "GeoLocation": None,  # Optional
    "Phone": phones,
    "CreatedAt": [datetime.utcnow() for _ in range(10)],
    "IsDeleted": 0
})

df_stores.to_csv("Stores.csv", index=False)
print(f"✅ Stores.csv created with {len(df_stores)} stores!")

✅ Stores.csv created with 10 stores!


C:\Users\NanoChip\AppData\Local\Temp\ipykernel_12920\3328075187.py:27: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  "CreatedAt": [datetime.utcnow() for _ in range(10)],
